# 감정분석 모델 탐색

하나하나 작업하기에는 데이터량이 많아서 우선 API 적용해보고 가장 적합한 API나 모듈 사용해보기

## naver api (CLOVA Sentiment)

In [171]:
import sys
import requests
import json
client_id = #아이디
client_secret = #시크릿
url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"
headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}
content = "무개념 개미들아"
data = {
  "content": content
}
print(json.dumps(data, indent=4, sort_keys=True))
response = requests.post(url, data=json.dumps(data), headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)

{
    "content": "\ubb34\uac1c\ub150 \uac1c\ubbf8\ub4e4\uc544"
}
{"document":{"sentiment":"negative","confidence":{"negative":99.9867,"positive":0.007990179,"neutral":0.0053068027}},"sentences":[{"content":"무개념 개미들아","offset":0,"length":8,"sentiment":"negative","confidence":{"negative":0.9992022,"positive":4.7941072E-4,"neutral":3.1840816E-4},"highlights":[{"offset":0,"length":8}]}]}


한국어는 결과가 꽤 정확하게 나오나 일일 사용량 제한이 있음.

## open ai (안 먹힘)

In [ ]:
import openai

openai.api_key = 'your-api-key'
def summarize_article(article_text):

    response = openai.Completion.create(
        engine="text-davinci-003",  # You can use other models like text-davinci-002 or other available engines
        prompt=f"Summarize the following article:\n\n{article_text}",
        max_tokens=150,  # Adjust the length of the summary
        temperature=0.5,
    )
    summary = response.choices[0].text.strip()
    return summary

article_text = """
    OpenAI provides powerful models for various NLP tasks, including text summarization. With the OpenAI Python module,
    you can easily interact with these models and perform tasks such as generating summaries, answering questions, and
    creating conversational agents. This capability is beneficial for applications in content creation, customer support,
    and information retrieval, among others. To use the OpenAI API, you need to install the openai Python package and
    obtain an API key from OpenAI's website. After setting up, you can make API calls to generate summaries and other
    types of content with just a few lines of code.
"""

summary = summarize_article(article_text)
print("Summary:", summary)

## TextBlob

textblob 써서 article content의 sentiment 점수 매기는 function이랑 긍/부/중 매기는 function 만들어서 column 추가?

In [183]:
from textblob import TextBlob

def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # Returns the polarity score: -1.0 to 1.0

def classify_sentiment(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'

def get_article_summary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    return article.summary

def main():
        for article in articles:
            combined_text = f"{title} {description} {content}"
            sentiment_score = analyze_sentiment(combined_text)
            sentiment = classify_sentiment(sentiment_score)

            print(f"Sentiment: {sentiment} (Score: {sentiment_score})")


## Transformers 사용

In [ ]:
import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
US_df = pd.read_excel("../data/clean_US_2020-2024.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
US_df['title'] = US_df['title'].fillna('')
US_df['content'] = US_df['content'].fillna('')
US_df['combined_text'] = US_df['title'] + ' ' + US_df['content']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='snunlp/KR-FinBert-SC', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    return result[0]['label']

# 감성 분석 및 새로운 열 생성
US_df['sentiment'] = US_df['combined_text'].apply(analyze_sentiment)

print(US_df[['title', 'content', 'sentiment']].head())

In [4]:
import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
US_df = pd.read_excel("../data/clean_US_2020-2024.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
US_df['title'] = US_df['title'].fillna('')
US_df['content'] = US_df['content'].fillna('')
US_df['combined_text'] = US_df['title'] + ' ' + US_df['content']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    return result[0]['label']

# 감성 분석 및 새로운 열 생성
US_df['sentiment'] = US_df['combined_text'].apply(analyze_sentiment)

print(US_df[['title', 'content', 'sentiment']].head())

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

                                               title  \
0                       Orthopedic Surgeon-Inventors   
1  A robot and human work together to remove tumo...   
2  Leading the Way in Robotic Surgery for Lung Ca...   
3     The patent power struggle in surgical robotics   
4  Cancer-Fighting Robot Awaits Medical Tourism P...   

                                             content sentiment  
0  Hackensack University Medical Center Orthopedi...   5 stars  
1  A robot, its human assistant and a team of sur...   4 stars  
2  Over the past decade, multiple studies have sh...   4 stars  
3  Guy Edwards, patent attorney at IP firm GJE, a...   4 stars  
4  Patients diagnosed with prostate cancer who ar...   4 stars  


In [5]:
import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
US_df = pd.read_excel("../data/clean_US_2020-2024.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
US_df['title'] = US_df['title'].fillna('')
US_df['content'] = US_df['content'].fillna('')
US_df['combined_text'] = US_df['title'] + ' ' + US_df['content']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    return result[0]['label']

# 감성 분석 및 새로운 열 생성
US_df['sentiment'] = US_df['combined_text'].apply(analyze_sentiment)

print(US_df[['title', 'content', 'sentiment']].head())

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

                                               title  \
0                       Orthopedic Surgeon-Inventors   
1  A robot and human work together to remove tumo...   
2  Leading the Way in Robotic Surgery for Lung Ca...   
3     The patent power struggle in surgical robotics   
4  Cancer-Fighting Robot Awaits Medical Tourism P...   

                                             content sentiment  
0  Hackensack University Medical Center Orthopedi...  POSITIVE  
1  A robot, its human assistant and a team of sur...  POSITIVE  
2  Over the past decade, multiple studies have sh...  POSITIVE  
3  Guy Edwards, patent attorney at IP firm GJE, a...  NEGATIVE  
4  Patients diagnosed with prostate cancer who ar...  POSITIVE  


In [12]:
import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
US_df = pd.read_excel("../data/clean_US_2020-2024.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
US_df['title'] = US_df['title'].fillna('')
US_df['content'] = US_df['content'].fillna('')
US_df['combined_text'] = US_df['title'] + ' ' + US_df['content']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return {'label': 'unknown', 'score': 0.0}  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment = result[0]['label']
    if sentiment == 'LABEL_0':
        score = -1  # Negative sentiment
    elif sentiment == 'LABEL_1':
        score = 0  # Neutral sentiment
    else:
        score = 1  # Positive sentiment
    return {'label': sentiment, 'score': score}

# 감성 분석 및 새로운 열 생성
US_df['sentiment'] = US_df['combined_text'].apply(lambda x: analyze_sentiment(x)['label'])
US_df['sentiment_score'] = US_df['combined_text'].apply(lambda x: analyze_sentiment(x)['score'])

# 결과를 CSV 파일로 저장
US_df.to_csv('sentiment_analysis_results.csv', index=False, encoding='utf-8-sig')

# 데이터프레임의 일부를 출력하여 결과를 확인
print(US_df[['title', 'content', 'sentiment', 'sentiment_score']].head())

                                               title  \
0                       Orthopedic Surgeon-Inventors   
1  A robot and human work together to remove tumo...   
2  Leading the Way in Robotic Surgery for Lung Ca...   
3     The patent power struggle in surgical robotics   
4  Cancer-Fighting Robot Awaits Medical Tourism P...   

                                             content sentiment  \
0  Hackensack University Medical Center Orthopedi...   LABEL_2   
1  A robot, its human assistant and a team of sur...   LABEL_2   
2  Over the past decade, multiple studies have sh...   LABEL_2   
3  Guy Edwards, patent attorney at IP firm GJE, a...   LABEL_1   
4  Patients diagnosed with prostate cancer who ar...   LABEL_1   

   sentiment_score  
0                1  
1                1  
2                1  
3                0  
4                0  


In [14]:
print(US_df.head())

         date                                              title  \
0  2020-01-03                       Orthopedic Surgeon-Inventors   
1  2020-01-03  A robot and human work together to remove tumo...   
2  2020-01-03  Leading the Way in Robotic Surgery for Lung Ca...   
3  2020-01-03     The patent power struggle in surgical robotics   
4  2020-01-04  Cancer-Fighting Robot Awaits Medical Tourism P...   

                                         description  \
0  Hackensack University Medical Center orthopedi...   
1  A robot its human assistant and a team of surg...   
2  NYU Langone Health is at the forefront of rese...   
3  Guy Edwards patent attorney at IP firm GJE ass...   
4  Patients diagnosed with prostate cancer who ar...   

                                                 url  \
0  https://www.hackensackmeridianhealth.org/info/...   
1  https://jagwire.augusta.edu/a-robot-and-human-...   
2  https://nyulangone.org/news/leading-way-roboti...   
3  https://www.med-technews.co

In [18]:
import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
US_df = pd.read_excel("../data/clean_US_2020-2024.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
US_df['title'] = US_df['title'].fillna('')
US_df['content'] = US_df['content'].fillna('')
US_df['combined_text'] = US_df['title'] + ' ' + US_df['content']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

# Sentiment mapping dictionary
sentiment_mapping = {
    '1 star': -2,
    '2 stars': -1,
    '3 stars': 0,
    '4 stars': 1,
    '5 stars': 2
}

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return {'label': 'unknown', 'score': 0.0}  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment = result[0]['label']
    score = sentiment_mapping.get(sentiment, 0)
    return {'label': sentiment, 'score': score}

# 감성 분석 및 새로운 열 생성
US_df['sentiment'] = US_df['combined_text'].apply(lambda x: analyze_sentiment(x)['label'])
US_df['sentiment_score'] = US_df['combined_text'].apply(lambda x: analyze_sentiment(x)['score'])

# 결과를 CSV 파일로 저장
US_df.to_csv('sentiment_analysis_results_multi.csv', index=False, encoding='utf-8-sig')

# 데이터프레임의 일부를 출력하여 결과를 확인
print(US_df[['title', 'content', 'sentiment', 'sentiment_score']].head())

                                               title  \
0                       Orthopedic Surgeon-Inventors   
1  A robot and human work together to remove tumo...   
2  Leading the Way in Robotic Surgery for Lung Ca...   
3     The patent power struggle in surgical robotics   
4  Cancer-Fighting Robot Awaits Medical Tourism P...   

                                             content sentiment  \
0  Hackensack University Medical Center Orthopedi...   5 stars   
1  A robot, its human assistant and a team of sur...   4 stars   
2  Over the past decade, multiple studies have sh...   4 stars   
3  Guy Edwards, patent attorney at IP firm GJE, a...   4 stars   
4  Patients diagnosed with prostate cancer who ar...   4 stars   

   sentiment_score  
0                2  
1                1  
2                1  
3                1  
4                1  


In [ ]:
print(US_df.head())

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

#data preprocessing
inputs = tokenizer("News article here", return_tensors="pt", max_length=512, truncation=True)


outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits)

print("predicted sentiment class:", predicted_class.item())

# US Surgical Robot 검색 데이터 감성분석

우선 일괄적용하기 전에 nlptown과 cardiffnlp 모델 얼마나 잘 분석하는지 비교

In [14]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_surgical robot_20-24/clean_US_2020-2024.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
nlptown_classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)
cardiffnlp_classifier = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment', device=device)

def nlptown_analyze(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = nlptown_classifier(text[:512])  # 모델의 최대 입력 길이 제한
    print(f"NLPtown result for text '{text[:50]}...': {result}")
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    return sentiment

def cardiffnlp_analyze(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = cardiffnlp_classifier(text[:512])  # 모델의 최대 입력 길이 제한
    print(f"CardiffNLP result for text '{text[:50]}...': {result}")
    sentiment_label = result[0]['label']

    if sentiment_label == 'LABEL_0':
        sentiment = 'negative'  # Negative sentiment
    elif sentiment_label == 'LABEL_1':
        sentiment = 'neutral' # Neutral sentiment
    else:
        sentiment = 'positive'  # Positive sentiment
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['nlptown_ctext'] = us_clean['combined_text'].apply(nlptown_analyze)
us_clean['nlptown_keyword'] = us_clean['keyword'].apply(nlptown_analyze)

us_clean['cardiffnlp_ctext'] = us_clean['combined_text'].apply(cardiffnlp_analyze)
us_clean['cardiffnlp_keyword'] = us_clean['keyword'].apply(cardiffnlp_analyze)


# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['nlptown_value'] = us_clean.apply(
    lambda row: 0 if row['nlptown_ctext'] == row['nlptown_keyword'] else 1, axis=1)

us_clean['cardiffnlp_value'] = us_clean.apply(
    lambda row: 0 if row['cardiffnlp_ctext'] == row['cardiffnlp_keyword'] else 1, axis=1)


NLPtown result for text 'Orthopedic Surgeon-Inventors Hackensack University...': [{'label': '5 stars', 'score': 0.41869866847991943}]
NLPtown result for text 'A robot and human work together to remove tumors f...': [{'label': '4 stars', 'score': 0.3760226368904114}]
NLPtown result for text 'Leading the Way in Robotic Surgery for Lung Cancer...': [{'label': '4 stars', 'score': 0.4794752895832062}]
NLPtown result for text 'The patent power struggle in surgical robotics Guy...': [{'label': '4 stars', 'score': 0.37877967953681946}]
NLPtown result for text 'Cancer-Fighting Robot Awaits Medical Tourism Patie...': [{'label': '4 stars', 'score': 0.41645365953445435}]
NLPtown result for text 'Medical Tourism: Lisbon Becomes Automatic Choice f...': [{'label': '4 stars', 'score': 0.4866374731063843}]
NLPtown result for text 'Microbot Medical raises $9M for disposable medical...': [{'label': '1 star', 'score': 0.47036299109458923}]
NLPtown result for text 'How the Versius robot could bring keyhole

In [15]:
us_clean[['date', 'title','content', 'keyword', 'nlptown_ctext', 'nlptown_keyword', 'cardiffnlp_ctext', 'cardiffnlp_keyword', 'nlptown_value', 'cardiffnlp_value']].to_csv("../data/US_surgical robot_20-24/US_surgical_senti_update.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'keyword', 'nlptown_ctext', 'nlptown_keyword', 'cardiffnlp_ctext', 'cardiffnlp_keyword', 'nlptown_value', 'cardiffnlp_value']].to_excel("../data/US_surgical robot_20-24/US_surgical_senti_update.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

CSV file created Successfully
Excel file created Successfully


파일 확인해보니 cardiffnlp는 SNS, 특히 twitter 특화 모델이라 그런지 웬만한 기사 키워드들은 죄다 neutral에 본문도 대부분 neutral. 그나마 잘 맞는게 nlptown인데 이마저도 사실 애매함. 본문을 봤을때 positive한 본문들을 오히려 neutral이라고 하고 오히려 neutral한 키워드들을 positive라고 봄. 대체 왜? 일단 nlptown 쓰자.

### US Surgical Robot Sentiment Analysis

In [6]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_surgical robot_20-24/clean_US_2020-2024.xlsx")

us_clean.dropna(subset=['title', 'description', 'url', 'content'], inplace=True)
us_clean = us_clean[~us_clean['date'].apply(lambda x: x == '1 month ago')]
us_clean = us_clean[~us_clean['keyword'].apply(lambda x: x == '' or x == [] or x == ['redirected'] or x == ['stuff'])]
us_clean = us_clean[~us_clean['content'].apply(lambda x: x == "© 2024 Google LLC, Sundar Pichai, 1600 Amphitheatre Parkway, Mountain View CA 94043, USA, 0807-882-594 (무료), yt-support-solutions-kr@google.com, 호스팅: Google LLC, 사업자정보 크리에이터들이 유튜브 상에 게시, 태그 또는 추천한 상품들은 판매자들의 약관에 따라 판매됩니다. 유튜브는 이러한 제품들을 판매하지 않으며, 그에 대한 책임을 지지 않습니다.")]
us_clean = us_clean[us_clean['content'].str.contains("Surgical robot" or "Medical Robot", case=False, na=False)]

us_clean = us_clean.sort_values("date")
us_clean = us_clean.set_index("date")
us_clean = us_clean.reset_index()


# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']
us_clean['link'] = us_clean['url']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['senti_ctext'] = us_clean['combined_text'].apply(analyze_sentiment)
us_clean['senti_keyword'] = us_clean['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['match_status'] = us_clean.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)


print(us_clean[['senti_ctext', 'senti_keyword', 'match_status']].head())


# 파일 저장시, 칼럼값 지정
us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_csv("../data/US_surgical robot_20-24/US_surgical_senti.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("../data/US_surgical robot_20-24/US_surgical_senti.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

  senti_ctext senti_keyword  match_status
0    positive      positive             0
1    positive      positive             0
2    positive      positive             0
3    positive      positive             0
4    positive      positive             0
CSV file created Successfully
Excel file created Successfully


### US Intuitive Surgical 검색 데이터 감성분석

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_sectors_20-24/intuitive_surgical_articles.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']
us_clean['link'] = us_clean['url']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['senti_ctext'] = us_clean['combined_text'].apply(analyze_sentiment)
us_clean['senti_keyword'] = us_clean['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['match_status'] = us_clean.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)


print(us_clean[['senti_ctext', 'senti_keyword', 'match_status']].head())


# 파일 저장시, 칼럼값 지정
us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_csv("intuitive_surgical_senti.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("intuitive_surgical_senti.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

### US Stryker Coporation 검색 데이터 감성분석

In [19]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_sectors_20-24/stryker_corporation_articles.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']
us_clean['link'] = us_clean['url']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['senti_ctext'] = us_clean['combined_text'].apply(analyze_sentiment)
us_clean['senti_keyword'] = us_clean['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['match_status'] = us_clean.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)


print(us_clean[['senti_ctext', 'senti_keyword', 'match_status']].head())


# 파일 저장시, 칼럼값 지정
us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_csv("../data/US_sectors_20-24/final/stryker_senti.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("../data/US_sectors_20-24/final/stryker_senti.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

  senti_ctext senti_keyword  match_status
0    positive      positive             0
1    positive      positive             0
2    negative      positive             1
3    positive      positive             0
4     neutral      positive             1
CSV file created Successfully
Excel file created Successfully


### US Medtronic PLC 검색 데이터 감성분석

In [10]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_sectors_20-24/medtronic_articles.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']
us_clean['link'] = us_clean['url']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['senti_ctext'] = us_clean['combined_text'].apply(analyze_sentiment)
us_clean['senti_keyword'] = us_clean['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['match_status'] = us_clean.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)


print(us_clean[['senti_ctext', 'senti_keyword', 'match_status']].head())


# 파일 저장시, 칼럼값 지정
us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_csv("../data/US_sectors_20-24/final/medtronic_senti.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("../data/US_sectors_20-24/final/medtronic_senti.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

  senti_ctext senti_keyword  match_status
0    positive      positive             0
1    positive      positive             0
2    positive      positive             0
3    negative      negative             0
4    negative      negative             0
CSV file created Successfully
Excel file created Successfully


### US Globus Medical 검색 데이터 감성분석

In [9]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_sectors_20-24/globus_medical_articles.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']
us_clean['link'] = us_clean['url']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['senti_ctext'] = us_clean['combined_text'].apply(analyze_sentiment)
us_clean['senti_keyword'] = us_clean['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['match_status'] = us_clean.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)


print(us_clean[['senti_ctext', 'senti_keyword', 'match_status']].head())


# 파일 저장시, 칼럼값 지정
us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_csv("../data/US_sectors_20-24/final/globus_medical_senti.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("../data/US_sectors_20-24/final/globus_medical_senti.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

  senti_ctext senti_keyword  match_status
0    positive      positive             0
1    positive      positive             0
2    positive      positive             0
3    positive      positive             0
4    negative      positive             1
CSV file created Successfully
Excel file created Successfully


### US Asensus surgical 검색 데이터 감성분석

In [7]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_sectors_20-24/asensus_surgical_articles.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']
us_clean['link'] = us_clean['url']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['senti_ctext'] = us_clean['combined_text'].apply(analyze_sentiment)
us_clean['senti_keyword'] = us_clean['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['match_status'] = us_clean.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)


print(us_clean[['senti_ctext', 'senti_keyword', 'match_status']].head())


# 파일 저장시, 칼럼값 지정
us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_csv("../data/US_sectors_20-24/asensus_surgical_senti.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("../data/US_sectors_20-24/asensus_surgical_senti.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

  senti_ctext senti_keyword  match_status
0    positive      positive             0
1    positive      positive             0
2    positive      positive             0
3    negative      positive             1
4    positive      positive             0
CSV file created Successfully
Excel file created Successfully


### US Smith & Nephew 검색 데이터 감성분석

In [8]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_sectors_20-24/smith_nephew_articles.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']
us_clean['link'] = us_clean['url']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['senti_ctext'] = us_clean['combined_text'].apply(analyze_sentiment)
us_clean['senti_keyword'] = us_clean['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['match_status'] = us_clean.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)


print(us_clean[['senti_ctext', 'senti_keyword', 'match_status']].head())


# 파일 저장시, 칼럼값 지정
us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_csv("../data/US_sectors_20-24/final/smith_nephew_senti.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("../data/US_sectors_20-24/final/smith_nephew_senti.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

  senti_ctext senti_keyword  match_status
0     neutral      positive             1
1    positive      positive             0
2    positive      positive             0
3    negative      positive             1
4    positive      positive             0
CSV file created Successfully
Excel file created Successfully


### US Johnson & Johnson 검색 데이터 감성분석

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
# Continue with your other imports and script logic

import pandas as pd
from transformers import pipeline
import torch

# CSV 파일 읽기
us_clean = pd.read_excel("../data/US_sectors_20-24/johnson_johnson_articles.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
us_clean['title'] = us_clean['title'].fillna('')
us_clean['content'] = us_clean['content'].fillna('')
us_clean['combined_text'] = us_clean['title'] + ' ' + us_clean['content']
us_clean['link'] = us_clean['url']

# GPU 사용 가능 여부 확인
device = 0 if torch.cuda.is_available() else -1

# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', device=device)

def analyze_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 'Unknown'  # 빈 문자열이나 NaN 값에 대한 처리
    result = classifier(text[:512])  # 모델의 최대 입력 길이 제한
    sentiment_label = result[0]['label']

    # Convert sentiment label to positive, negative, or neutral
    if sentiment_label in ['1 star', '2 stars']: #1star(-2), 2star(-1)
        sentiment = 'negative'
    elif sentiment_label == '3 stars': #3star(0)
        sentiment = 'neutral'
    else: #4 star(1) and 5 star(2)
        sentiment = 'positive'
    
    return sentiment

# 감성 분석 및 새로운 열 생성
us_clean['senti_ctext'] = us_clean['combined_text'].apply(analyze_sentiment)
us_clean['senti_keyword'] = us_clean['keyword'].apply(analyze_sentiment)

# senti_value 칼럼 생성, sentiment_from_ctext와 sentiment_from_keyword이 같으면, 0, 다르면 1
us_clean['match_status'] = us_clean.apply(
    lambda row: 0 if row['senti_ctext'] == row['senti_keyword'] else 1, axis=1)


print(us_clean[['senti_ctext', 'senti_keyword', 'match_status']].head())


# 파일 저장시, 칼럼값 지정
us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_csv("../data/US_sectors_20-24/johnson_johnson_senti.csv", index=False, encoding='utf-8')
print("CSV file created Successfully")

us_clean[['date', 'title','content', 'link', 'keyword', 'senti_ctext', 'senti_keyword', 'match_status']].to_excel("../data/US_sectors_20-24/johnson_johnson_senti.xlsx", index=False, engine='openpyxl')
print("Excel file created Successfully")

## 직접 학습시키는 모델

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load your dataset
data = pd.read_excel("../data/US_surgical robot_20-24/clean_US_2020-2024.xlsx")

# title과 content 열 합치기, NaN 값을 빈 문자열로 대체
data['title'] = data['title'].fillna('')
data['content'] = data['content'].fillna('')
data['combined_text'] = data['title'] + ' ' + data['content']

train_texts, val_texts, train_labels, val_labels = train_test_split(data['text'], data['label'], test_size=0.2)

# Create a Dataset object
train_dataset = Dataset.from_pandas(pd.DataFrame({'text': train_texts, 'label': train_labels}))
val_dataset = Dataset.from_pandas(pd.DataFrame({'text': val_texts, 'label': val_labels}))

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # Adjust num_labels based on your data

# Tokenize the data
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


In [ ]:
from transformers import pipeline

# Load the fine-tuned model
fine_tuned_model = BertForSequenceClassification.from_pretrained('./results')
fine_tuned_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create sentiment analysis pipeline
custom_classifier = pipeline('sentiment-analysis', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Example usage
text = "The new robotic system has improved efficiency in the manufacturing process."
result = custom_classifier(text)
print(result)